<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_1_SRAL_advanced_data_access_eumdac.ipynb" target="_blank"><< Advanced SRAL data access with the EUMDAC client</a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Ben Loveday (EUMETSAT/Innoflair UG), Hayley Evers-King (EUMETSAT), Vinca Rosmorduc (CLS)

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-sral/HEAD?urlpath=%2Ftree%2F2_SRAL_advanced%2F2_2_acquire_SRAL_full_cycle.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn SRAL: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
Users should review the following notebooks for more information on setting up a credentials files for using the eumdac python library to retrieve SRAL data from the Data Store:
    
- **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store.ipynb</a>**

</div>
<hr>

# 2.2 Acquiring a full SRAL cycle

### Data used


| Product Description  | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:--------:|
| Sentinel-3 SRAL level-2 | EO:EUM:DAT:0415 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SR_2_WAT___NTC?query=SRAL&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SR_2_WAT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_2_WAT___" target="_blank">link</a>|

### Learning outcomes

At the end of this notebook you will know;
* <font color="#138D75">**Search**</font> for a time series of L2 SRAL data for a given cycle number using the EUMETSAT Data Store API client (`eumdac`)
* <font color="#138D75">**Download**</font> this time series

### Outline

The EUMETSAT Data Store offers many ways to interact with data in order to refine searches. Many of these methods are supported by the EUMETSAT Data Access Client (`eumdac`). In this notebook we will showcase some of the possibilities for using `eumdac` to better interact with SRAL collections. This notebook assumes that you already have an understanding of the available Data Store interfaces, which you can gain by running the **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>** notebook.


<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
1. [Step 1: Authenticating the API](#section1)
1. [Step 2: Creating a search by collection](#section2)
1. [Step 3: Filtering by cycle number](#section3)
1. [Step 4: Filtering by timeliness](#section4)
1. [Step 5: Removing NTC duplicates](#section5)
1. [Step 6: Downloading the data](#section6)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
import os        # a library that allows us access to basic operating system commands
import json      # a library that helps us make JSON format files
import datetime  # a libary that allows us to work with dates and times
import shutil    # a library that allows us access to basic operating system commands like copy
import zipfile   # a library that allows us to unzip zip-files.
import eumdac    # a tool that helps us download via the eumetsat/data-store

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Step 1: Authenticating the API
[Back to top](#TOC-TOP)

</div>

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token. More **essential** information on setting this up can be found in the **<a href="../1_OLCI_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>** notebook.

In [3]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '0125bcd1-2012-30b9-a646-eca993a5f2f0' expires 2023-07-05 12:11:50.615168


<div class="alert alert-info" role="alert">

## <a id='section2'></a>Step 2: Creating a search by collection
[Back to top](#TOC-TOP)

</div>

We will work with the SRAL Level-2 data throughout this notebook. The collection ID for this data type is `EO:EUM:DAT:0415`. You can find this information on the Data Store (https://data.eumetsat.int/), or ask the `eumdac` client to tell you all the avaiable collections by calling the `eumdac.DataStore(token).collections` method.

In [4]:
# set collection ID for SRAL L2
collectionID = 'EO:EUM:DAT:0415'
cycle = 80

To filter by collection, we simply provide the collectionID to the `datastore.get_collection method`

In [5]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(f"{selected_collection.title}\n---\n{selected_collection.abstract}")

SRAL Level 2 Altimetry Global - Sentinel-3
---
The products contain the typical altimetry measurements, like the altimeter range, the sea surface height, the wind speed, significant wave height and all required geophysical corrections and related flags. Also the sea Ice freeboard measurement is included. The measurements in the standard data file provide the measurements in low (1 Hz = approx. 7km) and high resolution (20 Hz = approx. 300 m), in LRM mode or in SAR mode, for both C-band and Ku band. The SAR mode is the default mode. The reduced measurement data file contains 1 Hz measurements only. The enhanced measurement data file contains also the waveforms and associated parameters and the pseudo LRM measurements when in SAR mode. This product contains the following datasets: Sea Level Global(NRT) (PDS_MG3_CORE_14_GLONRT), Sea Level Global Reduced(NRT)(PDS_MG3_CORE_14_GLONRT_RD), Sea Level Global Standard(NRT) (PDS_MG3_CORE_14_GLONRT_SD), Sea Level Global Enhanced(NRT) (PDS_MG3_CORE

<div class="alert alert-info" role="alert">

## <a id='section3'></a>Step 3: Filtering by cycle number
[Back to top](#TOC-TOP)

</div>

Now that we have our collection we can search within it using the cycle number to filter the results, as follows;

In [6]:
products = selected_collection.search(
    cycle=cycle)
print(f"Found {len(products)} products")

Found 4847 products


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Step 4: Filtering by timeliness
[Back to top](#TOC-TOP)

</div>

In this instance, we are only interested in Non Time-Critical (NTC) SRAL products. We can add this filter to our search as follows.

In [7]:
products = selected_collection.search(
    cycle=cycle,
    timeliness="NT")
print(f"Found {len(products)} products")

Found 1221 products


<div class="alert alert-info" role="alert">

## <a id='section5'></a>Step 5: Removing NTC duplicates
[Back to top](#TOC-TOP)

</div>

Sometimes, during NTC processing, we have to revisit some granules to process them further. This is typically due to missing ancillary data that we have to wait for. In this case, we first pass and final pass products are both on the system. We only want the latter, final, product, which always has a later time. The box below will filter for the latest options only, taking advantage of the Data Stores default to show most recent products first.

In [8]:
processed_list = []
final_products = []
for product in products:
    file_tags = str(product).split('_')
    file_tags = [i for i in file_tags if i]
    granule_start = file_tags[4]
    if granule_start not in processed_list:
        final_products.append(product)
        processed_list.append(granule_start)
        
print(f"Found {len(final_products)} products")

Found 1221 products


<div class="alert alert-info" role="alert">

## <a id='section6'></a>Step 6: Downloading the products
[Back to top](#TOC-TOP)

</div>

We have now narrowed our search to a the required products. We can now download them using the code below to write them into our specified directory and unzip them.

In [9]:
max_products = 5
for final_product, count in zip(final_products, range(len(final_products))):
    if count >= max_products:
        continue
    print(f"Download {count}: {final_product}")
    with final_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(final_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {fdst.name} finished.')

    os.remove(fdst.name)

Download 0: S3B_SR_2_WAT____20230609T144142_20230609T152607_20230705T090059_2665_080_224______MAR_O_NT_005.SEN3
Download of product S3B_SR_2_WAT____20230609T144142_20230609T152607_20230705T090059_2665_080_224______MAR_O_NT_005.SEN3.zip finished.
Unzipping of product /Users/benloveday/Code/Git_Reps/CMTS/internal/sensors/learn-sral/2_SRAL_advanced/products/S3B_SR_2_WAT____20230609T144142_20230609T152607_20230705T090059_2665_080_224______MAR_O_NT_005.SEN3.zip finished.
Download 1: S3B_SR_2_WAT____20230609T134509_20230609T143213_20230705T075446_2824_080_224______MAR_O_NT_005.SEN3
Download of product S3B_SR_2_WAT____20230609T134509_20230609T143213_20230705T075446_2824_080_224______MAR_O_NT_005.SEN3.zip finished.
Unzipping of product /Users/benloveday/Code/Git_Reps/CMTS/internal/sensors/learn-sral/2_SRAL_advanced/products/S3B_SR_2_WAT____20230609T134509_20230609T143213_20230705T075446_2824_080_224______MAR_O_NT_005.SEN3.zip finished.
Download 2: S3B_SR_2_WAT____20230609T125646_20230609T13450

<div class="alert alert-block alert-warning">

### Challenge:

Now you have run this for acquiring the SRAL L2 data using the cycle number, can you adapt it for other orbital parameters?

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_1_SRAL_advanced_data_access_eumdac.ipynb" target="_blank"><< Advanced SRAL data access with the EUMDAC client</a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean" target="_blank">View on GitLab</a> | <a href="https://training.eumetsat.int/" target="_blank">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int target="_blank">Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int target="_blank">Contact our training team to collaborate on and reuse this material</a></span></p>